# Nearest Neighbors Algorithm for Supervised Learning


Nearest neighbors algorithms (NNAs) are very simple conceptually: to classify a datum with specific feature values, find the data point that has the most similar feature values and put the original datum in that class. NNAs can also be used to predict missing feature values. 

The most common NNA is the k-Nearest Neighbors algorithm where the top *K* nearest neighbors to the query are identified. In most instantiations of k-NNA, classification/prediction is based on a "majority vote" of the k nearest neighbors (ex. if k = 5 and 3 out of the 5 nearest neighbors are class A and 2 are class B, the new data point will be classified as A). In the image below, using k = 1 would yield a class 1 classification while k = 3 would yield class 2.


![alt text](https://cdn-images-1.medium.com/max/1600/0*Sk18h9op6uK9EpT8.)


----

### A quick example to illustrate k-NNA:

If we want to classify a car as "cool" or "uncool" using the features *horsepower, number of seats,* and *manual (0) or automatic (1)*, our dataset might look like this:

*   150, 5, 0, uncool (2008 Honda Civic)  
*   320, 5, 0, cool (2011 Dodge Charger)
*   383, 3, 1, cool (1985 Chevy Blazer)
*   210, 7, 0, uncool (2001 Honda Odyssey)

Let's say we're trying to predict whether the 2017 Bugatti Veyron (1500hp, 2 seats, manual: 1) is cool or not. In the next cell the data is loaded. 





In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
cars_dict = {'2008 Honda Civic':    {'hp':150., 'seats':5., 'auto':0., 'cool':0}, 
             '2011 Dodge Charger' : {'hp':320., 'seats':5., 'auto':0., 'cool':1}, 
             '1985 Chevy Blazer':   {'hp':383., 'seats':3., 'auto':1., 'cool':1}, 
             '2001 Honda Odyssey':  {'hp':210., 'seats':7., 'auto':0., 'cool':0}, 
             '2017 Bugatti Veyron': {'hp':1500.,'seats':2., 'auto':1., 'cool':None}}

data = pd.DataFrame.from_dict(cars_dict,orient='index')
data

,hp,seats,auto,cool
1985 Chevy Blazer,383.0,3.0,1.0,1.0
2001 Honda Odyssey,210.0,7.0,0.0,0.0
2008 Honda Civic,150.0,5.0,0.0,0.0
2011 Dodge Charger,320.0,5.0,0.0,1.0
2017 Bugatti Veyron,1500.0,2.0,1.0,NaN


### Normalizing Data and Calculating Distance

In order to tell how "near" one datum is to another, we need a way to measure the distance between two data. One of the simplest ways to do this (and the way we will be doing it) is with simple Euclidian distance (formula below). Euclidean distance is the square root of the sum of the difference between each feature squared:

![alt text](https://i.stack.imgur.com/2y0bx.png.)

Some other approaches include Chi square distance and cosine distance (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4978658/). 

NOTE: By normalizing the data in this way we are assuming all features are equally important, but there are ways to weight some features more/less than others.

If we simply compute the Euclidean distance from one datum to another, values that are generally larger (like horsepower) will end up having a greater effect than the other smaller values. Because features with large values are not inherently more important for prediction than other values, we should normalize the data before we calculate distance. One quick and easy way to normalize data is to divide each datum by the maximum value in its category (i.e. divide each element in a row by the max value in that row).

In [4]:
# Normalize the data by dividing each value by the maximum value in its row. 
# Do not normalize the labels indicating cool and uncool (row 3)

for i in ['hp','seats','auto']:
    ## your code here
    
    data[i] = data[i]/max(data[i])

data

,hp,seats,auto,cool
1985 Chevy Blazer,0.255333,0.428571,1.0,1.0
2001 Honda Odyssey,0.140000,1.000000,0.0,0.0
2008 Honda Civic,0.100000,0.714286,0.0,0.0
2011 Dodge Charger,0.213333,0.714286,0.0,1.0
2017 Bugatti Veyron,1.000000,0.285714,1.0,NaN


We can also use sklearn's [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) method to standardize the data.

Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual feature do not more or less look like standard normally distributed data (e.g. Gaussian with 0 mean and unit variance).

In [6]:
# Normalize data by removing the mean and scaling to unit variance from each feature

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_unitnorm = pd.DataFrame.from_dict(cars_dict, orient = "index")

for i in ['hp','seats','auto']:
    ## your code here
    
    feature_data = data_unitnorm[i].values.reshape(-1,1)
    scaler.fit(feature_data)
    data_unitnorm[i] = scaler.transform(feature_data)
    
data_unitnorm
    
    
    
    

,hp,seats,auto,cool
1985 Chevy Blazer,-0.259004,-0.802955,1.224745,1.0
2001 Honda Odyssey,-0.604742,1.491202,-0.816497,0.0
2008 Honda Civic,-0.724651,0.344124,-0.816497,0.0
2011 Dodge Charger,-0.384908,0.344124,-0.816497,1.0
2017 Bugatti Veyron,1.973305,-1.376494,1.224745,NaN


Now create a function that will calculate the euclidean distance between two data points (you can assume the data points will be arrays of the same length). 

In [7]:
# Distance function using formula for euclidean distance

def euclidean_dist(datum1, datum2):
  

  
    ## your code here
    innerval = 0
    for i in range(datum1.shape[0]):
      innerval = innerval + (datum1[i] - datum2[i])**2
    
    distance = math.sqrt(innerval)
    return(distance)
  

In the cell below, calculate the distance between the Bugatti and each other car using euclidean_dist. Remember not to use row three (cool/uncool label) when computing distance.



In [30]:
# FYI: This is how you can call a specific row by name and sub-select features
bugatti = data.loc["2017 Bugatti Veyron"][["hp","seats","auto"]].values
print(bugatti)

bugattinorm = data_unitnorm.loc["2017 Bugatti Veyron"][["hp","seats","auto"]].values
print(bugattinorm)

print("")

for i in list(data.index):
  d = euclidean_dist(bugatti, data.loc[i][["hp","seats","auto"]].values)
  
  print(d)
  
print("")
  
for j in list(data_unitnorm.index):
  x = euclidean_dist(bugattinorm, data_unitnorm.loc[j][["hp","seats","auto"]].values)
  
  print(x)

# for car in range(4):
#   d = euclidean_dist(bugatti, data.drop(['cool'], axis = 1).iloc[car,:)
#   print(d)


## your code here

[1.         0.28571429 1.        ]
[ 1.97330464 -1.3764944   1.22474487]

0.7582457436146612
1.4999346924558592
1.4119750243498486
1.3425788296529182
0.0

2.304809761974404
4.363103710427404
3.7955443494326357
3.5620726215795644
0.0


For the first version of normalization (dividing by the max), you should get the following distances (rounded to the thousandths palce):

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Blazer = 0.758

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Odyssey = 1.500

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Civic = 1.412

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Charger = 1.343

For the second version of normalization (standard norm), ou should get the following distances (rounded to the thousandths palce):

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Blazer = 2.305

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Odyssey = 4.363

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Civic = 3.796

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Charger = 3.562

Notice that (in this case) both normalization techniques yielded the same order of cars nearest-to-farthest.  This will not always be the case!

Because the distance between the Bugatti and Blazer is the smallest, if k = 1, we would classify the Bugatti as cool. If k = 4 we would not be able to classify the Bugatti in either category using the "majority vote" technique unless we had in place a tiebreaker protocol. Generally speaking, larger values of *k* reduce noise, but also make the boundaries between classes less distinct. The best value of *k* will depend on your dataset and your prediction needs.

# Predicting Diabetes in Pima Heritage Dataset

Next we will see if we can use k-NNA to predict whether or not a patient has diabetes given some medical information about them. Load and view the data in the cell below. 




In [35]:
# Data Loading and preprocessing

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
feature_names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age']

data = pd.read_csv(url, names=names)

#  'preg': number of pregnancies  
#  'plas': plasma glucose concentration 
#  'pres': blood pressure 
#  'skin': skin thickness
#  'test': Insulin
#  'mass': BMI
#  'pedi': diabetes pedigree function
#  'age': age
#  'class': '0' means does not have diabetes and '1' means has diabetes


# look at the first 10 rows
data.head(10)


,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


Now, let's clearly define which columns will act as explanatory variables, and which column will be the target value, and split the dataset between your training data and testing data.  Let's try an 80-20 split and use sklearn's [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) method (set random_state = 0 so we get the same output each time).

In [44]:
from sklearn.model_selection import train_test_split

x, y = data.loc[:, feature_names], data.loc[:,'class']
## Your code here

x_training, x_testing, y_training, y_testing = (train_test_split(x,y,test_size = .20, random_state=0))## Your code here

print(x_training.shape, y_training.shape)
print(x_testing.shape, y_testing.shape)

(614, 8) (614,)
(154, 8) (154,)


Let's not forget to normalize the data!

In [43]:
scaler = StandardScaler()

## Your code here


scaler.fit(x_training)
x_train = scaler.transform(x_training)

scaler.fit(y_training)
y_train = scaler.transform(y_training)



[[ 0.90832902  0.91569367  0.44912368 ...  0.37852648  0.67740401
   1.69955804]
 [ 0.03644676 -0.75182191 -0.47230103 ... -0.50667229 -0.07049698
  -0.96569189]
 [-1.12606292  1.38763205  1.06340683 ...  2.54094063 -0.11855487
  -0.88240283]
 ...
 [ 0.03644676 -0.84620959 -0.21634972 ... -0.94927168 -0.95656442
  -1.04898095]
 [ 2.0708387  -1.12937261  0.24436264 ... -0.26640405 -0.50001442
   0.11706589]
 [ 0.32707418  0.47521786  0.65388473 ... -4.07275877  0.52121586
   2.94889395]]


This predict method needs to compute the euclidean distance between the “new” observation and all the data points in the training set. Then, it assigns the corresponding label to the observation. Finally, it selects the K nearest ones and performs a "majority vote."



In [0]:
def predict(x_training, y_training, x_test_sample, k):
    
    ## Your code here


In [0]:
def knn(x_training, y_training, x_testing, k):
    
    ## Your code here

In [0]:
start = time.time()
predictions_slow = knn(x_training, y_training, x_testing, k=5)

print('Took {} seconds'.format(time.time() - start))
predictions_slow

# Using sklearn to speed up normalizing and distance finding

Luckily for us, sklearn has some quick and easy functions for normalizing the data, finding Euclidean distances, training, and testing with k-NNA. Try k = 5 to start.

In [45]:
from sklearn.neighbors import KNeighborsClassifier

# Create Model with k nearest neighbors
knn = KNeighborsClassifier(n_neighbors=5)

# Train/fit model with training data
knn.fit(x_training, y_training)

# Make predictions on the test data using the fitted model
start = time.time()
predictions_fast = knn.predict(x_testing)

print('Took {} seconds'.format(time.time() - start))
predictions_fast

Took 0.0016450881958007812 seconds


array([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Check sklearn's predictions and make sure they match yours.  Sklearn is faster, but you should get the same answers.

### Evaluating classification performance
Let's see how well our classifier did by looking at the confusion matrix and F1 score.

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

## Your code here

You should get F1 = 0.659. Notice that this classifier is pretty good at classifying negative samples (compare TN to FP), but we are not very good at classifying positive samples (compare TP to FN).

Spend a few minutes trying to increase the F1 score as much as you can by changing k and the features of the data you are using to predict values.

* What set of features and values of k did you find to be the most optimal?
* Why is choosing the right features so important for prediction accuracy? 
* What other model hyperparameters (other than k) might we be able to tune?
* How might we use non-numerical data columns in our model (if we had any)?

# Pros and Cons of k-NNA

## Pros
* Non-parametric (can be used with data that does not fit a normal distribution)
* Conceptually simple and relatively simple to instantiate
* Little to no "training" time 
* A good starting point/baseline classifier 

## Cons
* Slow "testing" phase compared to other predictors/classifiers 
* Degrades with high-dimension data (because there is less difference between closest and furthest neighbors)
* Unclear how to handle non-numeric features
* Doesn't handle data with skewed class distribution well (if one class is extremely dominant in the training data, it will dominate the "voting majority" for classifying new data)
* Features that are not of the same scale can (should) be normalized, but this introduces another model hyperparameter (i.e., which normalization method is appropriate?)
* Does not take into account feature correlation

---



From these exercises and the pros and cons listed above, here is when it is most useful to use k-NNA:
* Datasets with many data points and few dimensions (but can become very slow as well)
* Datasets that are non-parametric
* When you want a quick and easy classifier that does not have to be optimal (perhaps to use as a baseline for other models)


Lastly, there are many other types of NNAs and there are many other ways to instantiate the k-NNA. For example, we could have used a weighted voting system (nearer neighbors' votes carry more weight) instead of a majority voting system to classify. Additionally, we could have used a different distance measurement or a different error measurement. 